# Lecture 2 - continuation

# Tuple + asterisk = many fun

## Tuple unpacking  
  
Как мы помним, ```tuple``` - неизменяемый контейнер, хранящий каждый объект в заданной позиции  
Python позволяет нам "распаковывать" кортеж в разного рода присвоениях:

In [1]:
a, b = (1,2)
print(a+b)

songs = [
    ('In Flames', 'Clayman', 'Only for the Weak'),
    ('Deftones', 'White Pony', 'Change (In the House of Flies)'),
    ('Mastodon', 'Leviathan', 'Blood and Thunder')
]
for band, album, track in songs:
    print("Playing '%s' from the album '%s', composed by %s" % (track, album, band))

3
Playing 'Only for the Weak' from the album 'Clayman', composed by In Flames
Playing 'Change (In the House of Flies)' from the album 'White Pony', composed by Deftones
Playing 'Blood and Thunder' from the album 'Leviathan', composed by Mastodon


Более того, этот механизм работает на любых итерируемых значениях!  
Вариативность принимает одно значение, обозначенное звездочкой


In [2]:
first_name, *secondary_names, *last_name = ("Daniel", "Michael", "Blake", "Day-Lewis")

SyntaxError: two starred expressions in assignment (<ipython-input-2-5c84fcfbaf6a>, line 4)

In [3]:
first_name, *secondary_names, last_name = ("Benicio", "Monserrate", "Rafael", "del Toro Sánchez")
print(last_name)
print(secondary_names)

del Toro Sánchez
['Monserrate', 'Rafael']


In [4]:
item, *prices = ('Absolut', 803, 746, 1200)
print(prices)

[803, 746, 1200]


Как насчет вложенных значений?  
Без проблем, нужно задать присваивание на каждом уровне.  

In [5]:
songs = [
    ('In Flames', 'Clayman', 'Only for the Weak', (2000, 'Nuclear Blast')),
    ('Deftones', 'White Pony', 'Change (In the House of Flies)', (2000, 'Maverick Recording Company')),
    ('Mastodon', 'Leviathan', 'Blood and Thunder', (2004, 'Relapse'))
]
for band, album, track, (year, label) in songs:
    print("'%s' / '%s' -- %s => released in %d by label '%s'" % (track, album, band, year, label))

'Only for the Weak' / 'Clayman' -- In Flames => released in 2000 by label 'Nuclear Blast'
'Change (In the House of Flies)' / 'White Pony' -- Deftones => released in 2000 by label 'Maverick Recording Company'
'Blood and Thunder' / 'Leviathan' -- Mastodon => released in 2004 by label 'Relapse'


Вариативность также применяется на каждом уровне

In [6]:
band, album, track, (year, *label), *stuff = ('In Flames', 'Clayman', 'Only for the Weak', (2000, 'Nuclear Blast', 1,2,3), 7,8,9,)
label

['Nuclear Blast', 1, 2, 3]

In [7]:
stuff

[7, 8, 9]

## Function signatures  

Как мы помним с прошлой лекции, параметры для функции бывают позиционные и именованные.  

In [8]:
def func(pos_arg1, pos_arg2, kw_arg1=None, kw_arg2=None):
    pass

Python обладает удивительно элегантной обработкой параметров функции

In [9]:
def mega_function(a,b, *args, **kwargs):
    print("I do sum => %d" % (a+b))
    print("Have no idea what to do with that: %s" % str(args))
    print("And of course with those ones: %s" % str(kwargs))
    print("Guess I have received param 'k' = %s" % kwargs.get('k', 'No'))

mega_function(150, 150, 1,2,3,4,5, mode='tractor')

I do sum => 300
Have no idea what to do with that: (1, 2, 3, 4, 5)
And of course with those ones: {'mode': 'tractor'}
Guess I have received param 'k' = No


In [10]:
mega_function(150, 250, 1,2,3,4,5, mode='tractor', k='Yes')

I do sum => 400
Have no idea what to do with that: (1, 2, 3, 4, 5)
And of course with those ones: {'mode': 'tractor', 'k': 'Yes'}
Guess I have received param 'k' = Yes


Эластичная сигнатура добавляет гибкость, если не известен заранее объем входных параметров.  
Также это помогает обеспечить совместимость с функциями, которые зависит от определяемой функции.  
На мой взгляд, сигнатура вида ```(*args, **kwargs)``` может снизить наглядность использования

In [60]:
# Пример из реальной жизни
from sqlalchemy import create_engine
create_engine??

Начиная с Python 3.0, появился keyword-only стиль.  
Радость этого в том, что не надо выставлять keyword-параметру значение по умолчанию.  
Позиционные и ключевые параметры разделяются все той же звездочкой

In [13]:
def my_func(a, *, b, **kwargs):
    print(a+b)

my_func(1,b=2)

3


# Object  I  
  
(подробнее про объекты и классы у Ивана, 18.03)  
  
Все в Python является объектом.  
Даже нативные вещи вроде int, byte-строк.  
Почему?  

Подсказка - CPython разработан на C

Каждый объект имеет:
    1. идентичность (зависит от интерпретатора, в CPython - адрес в памяти)
    2. тип значения
    3. собственно само значение  
    4. счетчик ссылок
    
**Объект тождественен классу**  

Счетчик ссылок инкрементируется при каждом обращении.

Если счетчик ссылок обнуляется - память переменной помечается для удаления - **garbage collection**

# Memory layout  
  
После этой ячейки Ваша жизнь питониста никогда не будет прежней

## Stack and Heap  + PyMalloc
При запуске исполняемого файла, созданный процесс получает адресное пространство (виртуальная адресация в защищенном режиме CPU).  
https://www.kernel.org/doc/gorman/html/understand/understand007.html  
  
На большинcтве Unix-like системах полученное пространство разбивается на несколько секций:  
* Text - машинный код  
* Data - содержит инициазированные глобальные переменные  
* BSS - неявно инициализированные глобальные переменные (нули/NULL)  
* Stack - подчитывается при компиляции, на Linux - <= 8мб
* Heap - практически бесконечна. Аллокаторы памяти увеличивают пространство кучи вызывая brk()

![memlayout](./ipynb_content/memlayout.png)  
  
  
**Stack**:  
* LIFO-очередь. Каждый вызов создаюет новую запись в стеке - фрейм
* У каждого потока процесса есть свой стек
* Выделение памяти для объектов и последующее освобождение происходит автоматически  
* Работать с переменными в стеке - быстро (если сравнить с кучей)  
* Переменные в стеке жестко привязаны к размеру. Можно переполнить  
 
<img src="./ipynb_content/stackoverflow.jpg" alt="SO" width="700"/>
  
**Heap**  
* Протяженная область памяти  
* Получить оттуда память - через аллокатор (malloc,calloc,new,...)
* Нужно не забывать вернуть (free, ...)


**Python** хранит все объекты в выделенной зоне приватной кучи процесса интерпретатора. 
<img src="./ipynb_content/zones.png" alt="Zones" width="600"/>

Управление памятью - высокуровневое, каждый тип имеет свою стратегию выделения памяти.  
На нижних уровнях работает pymalloc  

<img src="./ipynb_content/hierarchy.png" alt="Mem hierarchy" width="700"/>  
  
**pymalloc** оптимизирован для выделения памяти < 512 байт.  
Для этого он разбивает память на арены (каждая по 256КБ).  
Арена в свою очередь состоит из пулов по 4КБ, а они уже из блоков (по 8, 16, 24, ... байт - класс хранения)  
```
Арена (256КБ)
|
|__Пул (4КБ)
   |
   |__Блок (зависит от категории)
```   

Каждый класс хранения оптимизирован под определенный внутренний тип данных - int4, int8, char ...  
  
В зависимости от обстоятельств pymalloc выберет нужную арену и свободные пулы.  
  
**Память CPython освобождает аренами. Поэтому если память плохо освобождается, она может начать 'протекать'**
(не так как в C/C++, но тоже ощутимо. Пока что ни один Garbage Collector не справится полностью с ними)
  
<img src="./ipynb_content/arena.png" alt="Arena" width="500"/>   

https://github.com/python/cpython/blob/master/Objects/obmalloc.c  
Подробный обзор по obmalloc.c
https://rushter.com/blog/python-memory-managment/  
   
Реализация списка:  
https://github.com/python/cpython/blob/master/Objects/listobject.c  
  
Пища для размышления и огромное спасибо за картинку  
https://stackoverflow.com/questions/18522574/cpython-memory-allocation




# Stack machine  
  
<img src="./ipynb_content/machine.gif" alt="SO" width="500"/>  
  
Многие интепретируемые языки (в том числе и **Python**) реализуют стек-машину.  
Принцип прост - на вершину стека в нужной последовательности закидываются объекты (переменные, функции, методы).  
Затем исполняем команду (**opcode**), которая забирает для себя нужное число операндов.  
Команда может вернуть итоговое значение на вершину стека.  
  
  
  
Каждый вложенный вызов порождает новый фрейм (элемент стека)

## Python bytecode  

Компиляция в байткод происходит:  
* собственно при запуске (python my_script.py)  
* при импорте компилируемого модуля (при этом сохраняется \*.pyc-файл)
* при вызове метода compile()

Для наших манипуляция отлично подойдет **compile**  
  
  
```compile(source, filename, mode, **kwargs)```  
  
* Код пользователя подается через source/filename
* ```mode``` - что компилируем? 
    * exec - последовательность выражений
    * eval - одно выражение
    * single - одно интерактивное выражение  
    
Нв выходе получим объект типа ```code```

In [14]:
src_code = """a = 123
b = 12
c = a+b
print("c is {}".format(c))
"""

ccode = compile(src_code, "", mode='exec')
type(ccode)

code

Самые полезные атрибуты объекта ```code```:    

|    Атрибут    |Назначение                                           |
| -------------- |----------------------------------------------------|
|```co_consts```|кортеж из констант, используемых конкретным байткодом|
|```co_names```|кортеж из глобальных переменных|
|```co_varnames```|кортеж из локальных переменных|
|```co_code```|собственно сам байткод|


In [56]:
ccode.co_consts

(123, 12, 'c is {}', None)

In [57]:
ccode.co_names

('a', 'b', 'c', 'print', 'format')

In [58]:
ccode.co_varnames

()

In [59]:
ccode.co_code

b'd\x00Z\x00d\x01Z\x01e\x00e\x01\x17\x00Z\x02e\x03d\x02\xa0\x04e\x02\xa1\x01\x83\x01\x01\x00d\x03S\x00'

Давайте раскодируем байткод!  


In [15]:
# Мы поговорим за генераторы чуть позже
import dis 

def unpack_op(bytecode): 
    extended_arg = 0
    for i in range(0, len(bytecode), 2):
        opcode = bytecode[i]
        if opcode >= dis.HAVE_ARGUMENT:
            oparg = bytecode[i+1] | extended_arg
            extended_arg = (oparg << 8) if opcode == dis.EXTENDED_ARG else 0
        else:
            oparg = None
        yield (i, opcode, oparg)

In [16]:
for offset, opcode, oparg in unpack_op(ccode.co_code):
    arg = 'None' if oparg is None else oparg
    outs = '{:>2}  {:>3}  {:>5}'.format(offset, opcode, arg)
    print(outs)

 0  100      0
 2   90      0
 4  100      1
 6   90      1
 8  101      0
10  101      1
12   23   None
14   90      2
16  101      3
18  100      2
20  160      4
22  101      2
24  161      1
26  131      1
28    1   None
30  100      3
32   83   None


In [17]:
for offset, opcode, oparg in unpack_op(ccode.co_code):
    arg = 'None' if oparg is None else oparg
    outs = '{:>2}  {:>15}  {:>5}'.format(offset, dis.opname[opcode], arg)
    print(outs)

 0       LOAD_CONST      0
 2       STORE_NAME      0
 4       LOAD_CONST      1
 6       STORE_NAME      1
 8        LOAD_NAME      0
10        LOAD_NAME      1
12       BINARY_ADD   None
14       STORE_NAME      2
16        LOAD_NAME      3
18       LOAD_CONST      2
20      LOAD_METHOD      4
22        LOAD_NAME      2
24      CALL_METHOD      1
26    CALL_FUNCTION      1
28          POP_TOP   None
30       LOAD_CONST      3
32     RETURN_VALUE   None


**Описание использованных опкодов**   
https://docs.python.org/3/library/dis.html#python-bytecode-instructions  
  
  **TOS** (top of stack)  
  Приcвоить TOS - закинуть элемент на вершину стека


|opcode|description|arguments|  
|------|-----------|---------|
|LOAD_CONST|TOS = константа из ```co_consts```|индекс константы
|LOAD_NAME|TOS = значение переменной из ```co_names```|индекс переменной
|STORE_NAME|Переменная из ```co_names``` = TOS|индекс переменной
|BINARY_ADD|Сложение двух верхних элементов стека|нет параметров
|CALL_FUNCTION|Вызов функции с указанием числа параметров|число параметров
|CALL_METHOD|Вызов метода объекта с указанием числа параметров|число параметров
|POP_TOP|удаление текущего TOS|
|RETURN_VALUE|вернуть текущий TOS родительскому вызову

Байт-код CPython, в отличии от Java Virtual Machine, не совместим между разными версиями.  
Но это не беда, Python - по настоящему интепретируемый ЯП!  
  
Чтобы получить красивую декомпиляцию со всеми тонкостями используемой версии, воспользуемся модулем **dis** из стандартной библиотеки

In [18]:
# исходная нумерация строк
for i,s in enumerate(src_code.split("\n")):
    print("{}   {}".format(i,s))

0   a = 123
1   b = 12
2   c = a+b
3   print("c is {}".format(c))
4   


In [19]:
import dis
from dis import dis as disasm

# dis сохраняет порядок строк
disasm(src_code)

print("\n\n\n [TODO] Визуализация выполнения - на доске")

  1           0 LOAD_CONST               0 (123)
              2 STORE_NAME               0 (a)

  2           4 LOAD_CONST               1 (12)
              6 STORE_NAME               1 (b)

  3           8 LOAD_NAME                0 (a)
             10 LOAD_NAME                1 (b)
             12 BINARY_ADD
             14 STORE_NAME               2 (c)

  4          16 LOAD_NAME                3 (print)
             18 LOAD_CONST               2 ('c is {}')
             20 LOAD_METHOD              4 (format)
             22 LOAD_NAME                2 (c)
             24 CALL_METHOD              1
             26 CALL_FUNCTION            1
             28 POP_TOP
             30 LOAD_CONST               3 (None)
             32 RETURN_VALUE



 [TODO] Визуализация выполнения - на доске


# Objects II

Давайте уясним одну вещь.  
Переменная в Python (и любом современном языке) - указатель на объект (некая область в памяти).  
Переменная сама по себе не является контейнером для объектов  
  
<img src="./ipynb_content/sticker.png" alt="SO" width="700"/>  

Парадигмы передачи объектов: 

## Call by reference  
  
<img src="./ipynb_content/cbr.jpg" alt="SO" width="500"/>  

## Call by value  

<img src="./ipynb_content/cbv.jpg" alt="SO" width="500"/>  
  
  
Thanks to https://robertheaton.com/2014/02/09/pythons-pass-by-object-reference-as-explained-by-philip-k-dick/

## Variable Scopes  
  
В **Python** предусмотрено три ключевых слова, определяющих зону действия переменной:  
* global
* local
* nonlocal

### global

In [20]:
a = 1
def f1():
    print(a)
f1()

1


In [21]:
disasm("""a = 1
def f1():
    print(a)
f1()""")

  1           0 LOAD_CONST               0 (1)
              2 STORE_NAME               0 (a)

  2           4 LOAD_CONST               1 (<code object f1 at 0x1120648a0, file "<dis>", line 2>)
              6 LOAD_CONST               2 ('f1')
              8 MAKE_FUNCTION            0
             10 STORE_NAME               1 (f1)

  4          12 LOAD_NAME                1 (f1)
             14 CALL_FUNCTION            0
             16 POP_TOP
             18 LOAD_CONST               3 (None)
             20 RETURN_VALUE

Disassembly of <code object f1 at 0x1120648a0, file "<dis>", line 2>:
  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_GLOBAL              1 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


In [22]:
# изменим глобальную переменную
a = 1
def f1():
    global a
    a *= 1000
    print(a)
f1()

1000


In [23]:
disasm("""a = 1
def f1():
    global a
    a *= 1000
    print(a)
f1()""")

  1           0 LOAD_CONST               0 (1)
              2 STORE_GLOBAL             0 (a)

  2           4 LOAD_CONST               1 (<code object f1 at 0x112064270, file "<dis>", line 2>)
              6 LOAD_CONST               2 ('f1')
              8 MAKE_FUNCTION            0
             10 STORE_NAME               1 (f1)

  6          12 LOAD_NAME                1 (f1)
             14 CALL_FUNCTION            0
             16 POP_TOP
             18 LOAD_CONST               3 (None)
             20 RETURN_VALUE

Disassembly of <code object f1 at 0x112064270, file "<dis>", line 2>:
  4           0 LOAD_GLOBAL              0 (a)
              2 LOAD_CONST               1 (1000)
              4 INPLACE_MULTIPLY
              6 STORE_GLOBAL             0 (a)

  5           8 LOAD_GLOBAL              1 (print)
             10 LOAD_GLOBAL              0 (a)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
      

### local

In [24]:
a = 1
def f1():
    a = 2
    print(a)
f1()

2


In [25]:
disasm("""a = 1
def f1():
    a = 2
    print(a)
f1()""")

  1           0 LOAD_CONST               0 (1)
              2 STORE_NAME               0 (a)

  2           4 LOAD_CONST               1 (<code object f1 at 0x112064f60, file "<dis>", line 2>)
              6 LOAD_CONST               2 ('f1')
              8 MAKE_FUNCTION            0
             10 STORE_NAME               1 (f1)

  5          12 LOAD_NAME                1 (f1)
             14 CALL_FUNCTION            0
             16 POP_TOP
             18 LOAD_CONST               3 (None)
             20 RETURN_VALUE

Disassembly of <code object f1 at 0x112064f60, file "<dis>", line 2>:
  3           0 LOAD_CONST               1 (2)
              2 STORE_FAST               0 (a)

  4           4 LOAD_GLOBAL              0 (print)
              6 LOAD_FAST                0 (a)
              8 CALL_FUNCTION            1
             10 POP_TOP
             12 LOAD_CONST               0 (None)
             14 RETURN_VALUE


In [26]:
a = 1
def f1():
    print(a)
    a = 2
f1()

UnboundLocalError: local variable 'a' referenced before assignment

In [27]:
disasm("""a = 1
def f1():
    print(a)
    a = 2
f1()""")

  1           0 LOAD_CONST               0 (1)
              2 STORE_NAME               0 (a)

  2           4 LOAD_CONST               1 (<code object f1 at 0x1120e0030, file "<dis>", line 2>)
              6 LOAD_CONST               2 ('f1')
              8 MAKE_FUNCTION            0
             10 STORE_NAME               1 (f1)

  5          12 LOAD_NAME                1 (f1)
             14 CALL_FUNCTION            0
             16 POP_TOP
             18 LOAD_CONST               3 (None)
             20 RETURN_VALUE

Disassembly of <code object f1 at 0x1120e0030, file "<dis>", line 2>:
  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_CONST               1 (2)
             10 STORE_FAST               0 (a)
             12 LOAD_CONST               0 (None)
             14 RETURN_VALUE


### nonlocal and closure

In [63]:
# TODO - скользящее среднее -> иллюстрация на доске
def make_rolling_avg():
    count = 0
    total = 0
    
    def avg(new_val):
        count += 1
        total += new_val;
        return total / count
    
    return avg

In [64]:
avg = make_rolling_avg()
# TODO uncomment me
avg(1)

UnboundLocalError: local variable 'count' referenced before assignment

**Что не так с этой функцией?**  
|  
|  
|  
|  
  

Переменные ```count, total``` есть в скоупах обеих функций.  
Нужно, чтобы ```avg``` работал с внешними переменными.  
Они для нее ни локальные, ни глобальные.  
Что остается? **nonlocal**

In [65]:
def make_rolling_avg():
    count = 0
    total = 0
    
    def avg(new_val):
        nonlocal count, total
        count += 1
        total += new_val;
        return total / count
    
    return avg

In [66]:
avg2 = make_rolling_avg()
print(avg2(2), avg2(4))

2.0 3.0


In [67]:
disasm(avg2)

  7           0 LOAD_DEREF               0 (count)
              2 LOAD_CONST               1 (1)
              4 INPLACE_ADD
              6 STORE_DEREF              0 (count)

  8           8 LOAD_DEREF               1 (total)
             10 LOAD_FAST                0 (new_val)
             12 INPLACE_ADD
             14 STORE_DEREF              1 (total)

  9          16 LOAD_DEREF               1 (total)
             18 LOAD_DEREF               0 (count)
             20 BINARY_TRUE_DIVIDE
             22 RETURN_VALUE


LOAD_DEREF(i)

    Loads the cell contained in slot i of the cell and free variable storage. Pushes a reference to the object the cell contains on the stack.
  
Свободная переменная - локальная переменная внешней функции, запрашиваемая вложенной

**Вывод**  
```avg``` сохраняет доступ к внешним переменным, даже после того как породившая их функция была задана   
  
```avg``` - **замыкание**

## Copy.copy()

### Shallow copies

In [33]:
a = [1,2,[3,4]]
b = list(a)
a[2] is b[2]

True

<img src="./ipynb_content/shallow.png" alt="SO" width="500"/>  

### Deep copies

In [34]:
from copy import deepcopy
a = [1,2,[3,4]]
b = deepcopy(a)
a[2] is b[2]

False

In [35]:
def func(i):
    print(id(i))
    return 2+i

base = 1002300
print(id(base))
func(base)

4597363280
4597363280


1002302

https://robertheaton.com/2014/02/09/pythons-pass-by-object-reference-as-explained-by-philip-k-dick/

## Equality and identity  
  
В каждого объекта есть идентификатор. 
В CPython - адрес в памяти  
  
  
Идентичность проверяется по ```id()```
Равенство - ```==```

**Вопрос для внимательных** - почему None is None == True?

In [36]:
None is None

True

In [37]:
NotImplemented is NotImplemented

True

**Identity**

In [38]:
id(int(1237829))

4597357936

In [39]:
id(int(1237829))

4597363472

In [40]:
bigmac = ['tasty', 'delicious', 'crunchy']
evil = bigmac
normfood = bigmac

print("My identity is %d" % id(bigmac))
print(bigmac is evil)
print(normfood == bigmac)

My identity is 4592544896
True
True


In [41]:
romans_lived = {'italy'}
itailians_live = {'italy'}
romans_lived == itailians_live

True

![2vars1obj](./ipynb_content/2vars2obj.png)

**Хозяйке на заметку** - Отличный визуализатор работы интерпретатора:  
http://pythontutor.com/visualize.html#mode=edit

# Классы  
  
Задать свой класс в питоне - нет ничего проще.

In [42]:
class Animal:
    pass

кошак = Animal()
собакен = Animal()

Класс имеет атрибуты и методы.  
Атрибуты - ```per class / per instance```  
Что из них статичное?

In [43]:
class Animal:
    creature_type = 'mammal'

кошак = Animal()
собакен = Animal()

кошак.creature_type

'mammal'

In [44]:
Animal.creature_type = 'arthropods'

кошак.creature_type

'arthropods'

Создание класса происходит в два этапа.  
```__new__``` - получение экземпляра класса (instance)  
```__init__``` - инициализация полученного экземпляра  
  
Разница сразу видна в сигнатурах.  
```cls``` - указатель на класс  
```self``` - указатель на экземпляр

Все класс в Python происходят от ```object```

In [45]:
CAT_COUNTER = 0

class Cat():
    def __new__(cls, *args, **kwargs):
        global CAT_COUNTER
        CAT_COUNTER += 1
        return object.__new__(cls)
    
    def __init__(self, name):
        self.name = name

In [46]:
for i in range(4):
    Cat(i)

print(CAT_COUNTER)

4


**BTW - в Python нет приватных атрибутов, все по взрослому**

In [47]:
Cat('murzik').name

'murzik'

# Object III

Объект предоставляет свой протокол (проще говоря - набор функций).  
Если реализовать часть таких функций - объект может быть принят различными функциями Python  
Некоторые их этих функций: 

| Функция | Что делает |
|---------|------------|
|```__new__```|создание экземпляра класса|
|```__init__```|инициализация атрибутов класса
|```__del__```|финализатор класса (но не его деструктор)
|```__str__```|текстовое представление класса (используется в ```print```, ```format```)
|```__repr__```|текстовое представление класса "для разработчика"
|```__bytes__```|представление класса в байтах (не совсем строка)  
  
Методы, название которых имеет вид **__XYZ__** часто называются "магическими".  
Праивльно произношение - **dunder**.  Например: dunder-**init**

Методы всегда принимают первым параметром указатель на экземпляр

In [48]:
class Cat():
    def __init__(self, name):
        self.name = name
        

class CatStr():
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return "<Cat, named='{}'>".format(self.name)

In [49]:
print(Cat("murzik"))

In [50]:
print(CatStr("murzik"))

<Cat, named='murzik'>


## Collections 

Чтобы выступать в роли коллекции, объекту нужно предоставлять опеределенные методы.  
В типизированных языках (Java, C#) такое явление называют интерфейсом, в Python/Smalltalk - протоколом  
Протокол, в отличии от интерфейса, не является типом и выражает "договоренность"  


Фреймворк коллекций в Python формализован в стандартном модуле ```collections.abc```  
  
https://asvetlov.blogspot.com/2014/09/abstract-containers.html

In [1]:
%%html 
'<iframe src=./ipynb_content/collections.pdf width=800 height=1100></iframe>'

Рассмотрим один из базовых протоколов - **Sequence**

In [2]:
%%html 
'<iframe src=./ipynb_content/sequence.pdf width=500 height=300></iframe>'

In [156]:
data = [1,2,3]
print(len(data))
print(data.__len__())

3
3


In [3]:
from collections.abc import Sequence
from random import randint

In [4]:
class FakeCollection(Sequence):
    def __init__(self):
        self.length = randint(1,15)
        self.data = []
        for i in range(self.length):
            self.data.append(randint(1,100))
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        return self.data[index]

In [5]:
fc = FakeCollection()

In [6]:
# iter works
for d in fc:
    print(d)

100
29
73
41
44
51
46
61
71
9
89
58
50


In [7]:
# getitem works
fc[0]

100

In [8]:
# len also works
len(fc)

13

## Duck typing  
  
```Если это выглядит как утка, плавает как утка и крякает как утка, то это, вероятно, и есть утка. ```  
  
Пример с ```FakeCollection``` - объект реализовал метод **__iter__**, поэтому считается итерируемым (иначе говоря, реализовывает протокол итерации).  



# Домашнее задание  
  
  
Реализовать свой домашний ```ArrayList```   
В отличии от стандартного списка, ваш должен быть типизированным.  
Для проходного балла - класс должен реализовать протокол ```Sequence```  
Для мотивированных и любопытных - класс должен реализовать протокол ```MutableSequence```  
   
Каждый метод из протокола должен быть проверен.  
  
ДЗ - нужно сделать форк от нашего репозитория и прислать в Slack ссылку на Pull Request     

Hint 1 - изучите плоский массив: ```array.array```, внутреннее хранение должно быть на нем.  
Hint 2 - наследование от классов из ```collections.abc``` **не лопускается!**

In [17]:
from array import array
from math import floor

class ArrayList:
    def __init__(self, char, collection):
        self.__char = char
        self.__inner_array = array(char,collection)
        self.__list_counter = 0
        
    def __len__(self):
        counts = 0
        for elem in self.__inner_array:
            counts+=1
        return counts
    
    def __getitem__(self,index):
        if type(index) == slice:
            getitem_list = []
            for elem in self.__inner_array[index]:
                getitem_list+=[elem]
            return getitem_list
        else:
            return self.__inner_array[index]
    
    def __setitem__(self,index,value):
        self.__inner_array[index] = value
    
    def __delitem__(self, key):
        print("Deletes!")
        del self.__inner_array[key]
    
    def __contains__(self,item):
        return (item in self.__inner_array)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.__list_counter < self.__len__():
            self.__list_counter+=1
            return self.__inner_array[self.__list_counter-1]
        else:
            self.__list_counter = 0
            raise StopIteration
    
    def __reversed__(self):
        len = self.__len__()
        for i in range(floor(len/2)):
            elem = self.__inner_array[i]
            self.__inner_array[i] = self.__inner_array[len - i - 1]
            self.__inner_array[len - i - 1] = elem
        
    def insert(self,index,item):
        innerarray = self.__inner_array[0:index]
        innerarray+= array(self.__char,[item])
        innerarray+= self.__inner_array[index::]
        del self.__inner_array
        self.__inner_array = innerarray
        
    def index(self,item):
        index = 0
        for elem in self.__inner_array:
            if elem == item:
                return index
            index+=1
        return 0
    
    def count(self,item):
        counts = 0
        for elem in self.__inner_array:
            if elem == item:
                counts+=1
        return counts
    
    def append(self,item):
        self.__inner_array+=array(self.__char,[item])
        
    def reverse(self):
        self.__reversed__()
    
    def extend(self,item):
        self.__inner_array+=array(self.__char,item)
        
    def pop(self):
        number_be_dead = len(self)-1
        return_item = self.__inner_array[number_be_dead]
        del self.__inner_array[number_be_dead]
        return return_item
    
    def remove(self, value):
        for i in range(len(self.__inner_array)):
            if self.__inner_array[i] == value:
                del self.__inner_array[i]
                break

    def __iadd__(self,value):
        self.__inner_array+=array(self.__char,value)
        return self
                
    def return_it(self):
        return list(self.__inner_array)

this_arraylist = ArrayList("d",[0.3,0.4,0.21,0.4])
print("len = %d" %len(this_arraylist))
print("1st element = %d" %this_arraylist[1])
print("slice 1:3:1 = %s" %str(this_arraylist[1:3:1]))
print("contains %f is %s " % (0.4, str(0.4 in this_arraylist)))
print("contains %f is %s " % (0.1, str(0.1 in this_arraylist)))
print(gnome for gnome in this_arraylist)
for gnome in this_arraylist:
    print(gnome)
for gnome in this_arraylist:
    print(gnome)
print("index of %f = %s" % (0.4, str(this_arraylist.index(0.4))))
reversed(this_arraylist)
print("array after reverse %s" % str(this_arraylist.return_it()))
print("index of %f = %s" % (0.4, str(this_arraylist.index(0.4))))
print("counts of %f = %d" %(0.5, this_arraylist.count(0.5)))
print("counts of %f = %d" %(0.4, this_arraylist.count(0.4)))
print("counts of %f = %d" %(0.3, this_arraylist.count(0.3)))
this_arraylist[2] = 0.14
print("After set %s" %str(this_arraylist.return_it()))
del this_arraylist[1]
print("After deletes %s" %str(this_arraylist.return_it()))
def test_method(arraylist,method_to_do,*value_to_test):
    method_to_do(*value_to_test)
    print("After implement %s to %s result is %s" %(str(method_to_do.__name__),str(value_to_test),str(arraylist.return_it())))
test_method(this_arraylist,this_arraylist.append,0.68)
test_method(this_arraylist,this_arraylist.insert,1,0.56)
print("After inserts %f to %d %s" %(0.56,1.0,str(this_arraylist.return_it())))
this_arraylist.reverse()
print("After self reverse by reverse() %s" %str(this_arraylist.return_it()))
list_to_extend = [3.5,0.8]
test_method(this_arraylist,this_arraylist.extend,list_to_extend)
#test_method(["п","о","ф","1","g"],this_arraylist.extend)
#test_method(5,this_arraylist,this_arraylist.extend)
print("Pop result %s and array %s" %(str(this_arraylist.pop()),str(this_arraylist.return_it())))
print("Pop result %s and array %s" %(str(this_arraylist.pop()),str(this_arraylist.return_it())))
this_arraylist.remove(0.3)
print("remove %f result array %s" %(0.3,str(this_arraylist.return_it())))
list_to_test=[2.4,6.9]
this_arraylist+=list_to_test
print("iadd result %s and array %s" %(str(list_to_test),str(this_arraylist.return_it())))

len = 4
1st element = 0
slice 1:3:1 = [0.4, 0.21]
contains 0.400000 is True 
contains 0.100000 is False 
<generator object <genexpr> at 0x776b373f20>
0.3
0.4
0.21
0.4
0.3
0.4
0.21
0.4
index of 0.400000 = 1
array after reverse [0.4, 0.21, 0.4, 0.3]
index of 0.400000 = 0
counts of 0.500000 = 0
counts of 0.400000 = 2
counts of 0.300000 = 1
After set [0.4, 0.21, 0.14, 0.3]
Deletes!
After deletes [0.4, 0.14, 0.3]
After implement append to (0.68,) result is [0.4, 0.14, 0.3, 0.68]
After implement insert to (1, 0.56) result is [0.4, 0.56, 0.14, 0.3, 0.68]
After inserts 0.560000 to 1 [0.4, 0.56, 0.14, 0.3, 0.68]
After self reverse by reverse() [0.68, 0.3, 0.14, 0.56, 0.4]
After implement extend to ([3.5, 0.8],) result is [0.68, 0.3, 0.14, 0.56, 0.4, 3.5, 0.8]
Pop result 0.8 and array [0.68, 0.3, 0.14, 0.56, 0.4, 3.5]
Pop result 3.5 and array [0.68, 0.3, 0.14, 0.56, 0.4]
remove 0.300000 result array [0.68, 0.14, 0.56, 0.4]
iadd result [2.4, 6.9] and array [0.68, 0.14, 0.56, 0.4, 2.4, 6.9]


#  На десерт (по внутренностям)

* Воспроизвести патчинг Python (рекомендуется делать строго на виртуалке/docker под linux)  

https://eli.thegreenplace.net/2010/06/30/python-internals-adding-a-new-statement-to-python/

* Детали реализации питоновского спика

http://www.laurentluce.com/posts/python-list-implementation/